## 1. Get Id's from assigned requests

In [ ]:
#!pip install snowflake-connector-python
#!pip install tableauhyperapi
#!pip install tableau-api-lib

### Variables to modify

In [ ]:
import json

auth_path = r".\credentials.json"
auth_path = auth_path.replace("\\","/")

with open(auth_path) as json_file:
    json_credentials = json.load(json_file)
    json_file.close()

user = json_credentials["cisco_tableau"]["user"] # Your Cisco e-mail address
tableau_template_name = json_credentials["paths"]["tableau_template_oe_path"] #twb template name/path
ib_file_name = 'IB.csv'
coverage_file_name = 'Coverage.csv'
sw_file_name = 'SW.csv'
sntc_mapping_path = json_credentials["paths"]["sntc_mapping_path"]
personal_access_token_name = json_credentials["cisco_tableau"]["access_token_name"]
personal_access_token_secret = json_credentials["cisco_tableau"]["token"]

### Getting the accounts to work on

In [ ]:
import os
import datetime
import glob
import pandas as pd
import numpy as np
import shutil
import itertools
import xml.etree.ElementTree as ET
import snowflake.connector
import re
import webbrowser
import importlib
import smartsheet_lib as smartsheet
import utils_lib as utils
import requests

class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    
current_path = os.getcwd().replace("\\","/")

date = datetime.datetime.today()
date = date.date()

if date.weekday() in [0,1]:
    delta_t = str(date-datetime.timedelta(4))
else:
    delta_t = str(date-datetime.timedelta(1))
    
month = datetime.datetime.today().strftime("%B")

print ('Current Path: ' + str(current_path))
print ('Current Date: ' + str(date))

# -----------------------------------------------------------------


smartsheet_client = smartsheet.init_conn(json_credentials["smartsheet"]["API_access_token"])
#oa_sheet = smartsheet.load_sheet(7190965472520068,client=smartsheet_client,modified_since=delta_t)  #Q3
#oa_sheet = smartsheet.load_sheet(1763453217073028,client=smartsheet_client,modified_since=delta_t)  #Q4
oa_sheet = smartsheet.load_sheet(3815245262153604,client=smartsheet_client,modified_since=delta_t)  #Q4-2
oa_df = pd.DataFrame()
sheet_new = smartsheet.get_last_n_rows(oa_sheet,n_rows=4000)
oa_df = smartsheet.sheet_to_df2(sheet_new,columns=oa_sheet.columns)
oa_df = oa_df.query("`Request ID` != ''")
oa_df['Request ID'] = oa_df['Request ID'].apply(lambda x:int(x)) # drop request id decimal places

fields_df = utils.get_da_requests(da=user,df=oa_df)
fields_df

In [ ]:
fields_df

In [ ]:
# IDs list for CRBO queries (separator must be ';')
savs_list,gu_list,cav_list,cr_list = utils.get_ids_list(fields_df,separator=';')
cav_names = utils.get_cav_names(user,cav_list.replace(";",","))
print(f"SAVs: {savs_list}")
print(f"GUs: {gu_list}")
print(f"CAVs: {cav_list}")

print("CAV NAMEs:")
for idx in cav_names["CAV NAME"]:
    if idx == cav_names["CAV NAME"][len(cav_names["CAV NAME"])-1]:
        print(idx)
    else:
        print(idx,end=";")

print(f"CR Parties: {cr_list}")

## 2. Data from Snowflake

In [ ]:
# IDs list for Snowflake (separator must be ',')
savs_list,gu_list,cav_list,cr_list = utils.get_ids_list(fields_df,separator=',')


In [ ]:

# Telemery dataframe from Snowflake (may take a few minutes)
telemetry_df = pd.DataFrame()
telemetry_df = utils.get_telemetry_df2(user,savs_list,gu_list,cr_list,cav_list)
dna_df = utils.get_dna_df(user,savs_list,gu_list,cr_list,cav_list)
print(f"Telemetry data - Number of rows retrieved: {len(telemetry_df)}")

In [ ]:
    #if len(dna_df) > 0:
        #dna_df.to_csv(current_path + "/dna_df.csv",index=False)
        #print("DNA file created successfully")

In [ ]:
# TAC dataframe from Snowflake (may take a few minutes)
tac_df = pd.DataFrame()
tac_df_gu = pd.DataFrame()
tac_df_cav = pd.DataFrame()
tac_df_cr = pd.DataFrame()

if 'SAV ID' in fields_df["ID TYPE"].unique():
    tac_df = utils.get_tac_df_new(user=user,ids=savs_list,id_type='SAV ID')
    tac_df["BUG_CNT"] = tac_df["BUG_CNT"].replace("",0.0)
    print(f"SAV TAC data - Number of rows retrieved: {len(tac_df)}")
if 'GU ID' in fields_df["ID TYPE"].unique():
    tac_df_gu = utils.get_tac_df_new(user=user,ids=gu_list,id_type='GU ID')
    tac_df_gu["BUG_CNT"] = tac_df_gu["BUG_CNT"].replace("",0.0)
    print(f"GU TAC data - Number of rows retrieved: {len(tac_df_gu)}")
if 'CAV ID' in fields_df["ID TYPE"].unique():
    tac_df_cav = utils.get_tac_df_new(user=user,ids=cav_list,id_type='CAV ID')
    tac_df_cav["BUG_CNT"] = tac_df_cav["BUG_CNT"].replace("",0.0)
    print(f"CAV TAC data - Number of rows retrieved: {len(tac_df_cav)}")
if 'CR Party ID' in fields_df["ID TYPE"].unique():
    tac_df_cr = utils.get_tac_df_new(user=user,ids=cr_list,id_type='PARTY ID')
    tac_df_cr["BUG_CNT"] = tac_df_cr["BUG_CNT"].replace("",0.0)
    print(f"CR TAC data - Number of rows retrieved: {len(tac_df_cr)}")

## 3. Package creation

#### Note: Run the following lines only if you have already downloaded the data from CRBO and saved it in their respective SAV_ID/GU_ID folders

In [ ]:
# -------------------------------------------------------- Load SAV/GU files

renewals_s = pd.DataFrame()
coverage_s = pd.DataFrame()
subs_s = pd.DataFrame()
    
renewals_g = pd.DataFrame()
coverage_g = pd.DataFrame()
subs_g = pd.DataFrame()

renewals_c = pd.DataFrame()
coverage_c = pd.DataFrame()
subs_c = pd.DataFrame()
    
renewals_cr = pd.DataFrame()
coverage_cr = pd.DataFrame()
subs_cr = pd.DataFrame()

sntc_mapping = pd.read_excel(sntc_mapping_path,sheet_name="Output").fillna(0)

if 'SAV ID' in fields_df["ID TYPE"].unique():
    for f in os.listdir(f"{current_path}/CRBO/SAV_ID"):
        if f.__contains__(ib_file_name):
            df_s = pd.read_csv(f"{current_path}/CRBO/SAV_ID/{f}", 
                                 dtype={'SAV ID':str,
                                    'Instance Shipped Fiscal Year':str,
                                    'Item Quantity':float,
                                    'LDoS FY':str,
                                    'Best Site ID':float,
                                    'Contract Number':str,
                                    'Service Brand Code':str})
            df_s.insert(loc=2, column='Best Site ID', value=-9999)
            df_s.insert(loc=3, column='Best Site Customer Name', value='UNKNOWN')
            renewals_s = renewals_s.append(df_s)                             
    coverage_s = pd.read_csv(f"{current_path}/CRBO/SAV_ID/{coverage_file_name}", dtype={'SAV ID':str})
    subs_s = pd.read_csv(f"{current_path}/CRBO/SAV_ID/{sw_file_name}", dtype={'SAV ID':str})
    print('SAV files loaded!')
    print(len(renewals_s))
    
if 'GU ID' in fields_df["ID TYPE"].unique():
    for f in os.listdir(f"{current_path}/CRBO/GU_ID"):
        if f.__contains__(ib_file_name):
            df_g = pd.read_csv(f"{current_path}/CRBO/GU_ID/{f}", 
                                 dtype={'Best Site GU Party ID':str, 
                                        'Instance Shipped Fiscal Year':str,
                                        'Item Quantity':float,
                                        'LDoS FY':str,
                                        'Service Brand Code':str})
            df_g.insert(loc=2, column='Best Site ID', value=-9999.0)
            df_g.insert(loc=3, column='Best Site Customer Name', value='UNKNOWN')
            renewals_g = renewals_g.append(df_g)
    coverage_g = pd.read_csv(f"{current_path}/CRBO/GU_ID/{coverage_file_name}", dtype={'Best Site GU Party ID':str})
    subs_g = pd.read_csv(f"{current_path}/CRBO/GU_ID/{sw_file_name}", dtype={'GU Party ID':str})
    print('GU files loaded!')
    print(len(renewals_g))
    
if 'CAV ID' in fields_df["ID TYPE"].unique():
    for f in os.listdir(f"{current_path}/CRBO/CAV_ID"):
        if f.__contains__(ib_file_name):
            df_cav = pd.read_csv(f"{current_path}/CRBO/CAV_ID/{f}", 
                                 dtype={#'CX Customer ID':str, 
                                        'Instance Shipped Fiscal Year':str,
                                        'Item Quantity':float,
                                        'LDoS FY':str,
                                        'Service Brand Code':str})
            df_cav.insert(loc=1, column='Best Site ID', value=-9999)
            df_cav.insert(loc=2, column='Best Site Customer Name', value='UNKNOWN')
            renewals_c = renewals_c.append(df_cav)
    coverage_c = pd.read_csv(f"{current_path}/CRBO/CAV_ID/{coverage_file_name}") #dtype={'CX Customer ID':str}
    subs_c = pd.read_csv(f"{current_path}/CRBO/CAV_ID/{sw_file_name}") #dtype={'CX Customer ID':str}
    print('CAV files loaded!')
    print(len(renewals_c))
    
if 'CR Party ID' in fields_df["ID TYPE"].unique():
    for f in os.listdir(f"{current_path}/CRBO/PARTY_ID"):
        if f.__contains__(ib_file_name):
            df_cr = pd.read_csv(f"{current_path}/CRBO/PARTY_ID/{f}", 
                                 dtype={'Best Site CR Party ID':str, 
                                        'Instance Shipped Fiscal Year':str,
                                        'Item Quantity':float,
                                        'LDoS FY':str,
                                        'Service Brand Code':str})
            df_cr.insert(loc=2, column='Best Site ID', value=-9999)
            df_cr.insert(loc=3, column='Best Site Customer Name', value='UNKNOWN')
            renewals_cr = renewals_cr.append(df_cr)
    coverage_cr = pd.read_csv(f"{current_path}/CRBO/PARTY_ID/{coverage_file_name}", dtype={'Best Site CR Party ID':str})
    subs_cr = pd.read_csv(f"{current_path}/CRBO/PARTY_ID/{sw_file_name}", dtype={'CR Party ID':str})
    print('CR files loaded!')
    print(len(renewals_cr))

In [ ]:
smartsheet_fields = ['index','Request ID','Date Created',
                         'Assigned DA','Campaign Name','Customer Name',
                         'Input file URL','ID TYPE','SAV ID','CAV ID',
                         'CAV BU ID','GU ID','Lvl1','Lvl2 (Region)',
                         'Contract ID','Inventory Name','Appliance ID',
                         'CR Party Name','CR Party ID','Comments','DA Comments',
                         'Status','Requester Name','Who should be notified on completion of Analysis',
                         'OP Status']
folders_list = []
folders_path_list = []
type_list = []
sntc_oppty_list = []
renewals = pd.DataFrame()
coverage = pd.DataFrame()
subs = pd.DataFrame()
map_req_type = {'SAV ID':'SAV','GU ID':'GU','CR Party ID':'CR','CAV ID':'CAV'}

# ------------------------------------------------------------------- Mapping Files

local_path = "."

contract_types_list = pd.read_excel(f"{local_path}/Mapping files/1-Contract_Types_List.xlsx", sheet_name='SNTCSolutionSupport')
contract_types_list = contract_types_list.astype({'Contract Type':str},errors='raise')

success_track_pricing_list = pd.read_excel(f"{local_path}/Mapping files/2-Success_Track_Pricing_List.xlsx", sheet_name='Success Track PIDs')
success_track_pricing_list = success_track_pricing_list.astype({'Product SKU':str},errors='raise')

sspt_pricing_list_eligibleSSPT = pd.read_excel(f"{local_path}/Mapping files/3-SSPT_Princing_List.xlsx", sheet_name='Eligible SSPT')
sspt_pricing_list_eligibleSSPT = sspt_pricing_list_eligibleSSPT.astype({'Product SKU':str},errors='raise')

sspt_pricing_list_outputTable = pd.read_excel(f"{local_path}/Mapping files/3-SSPT_Princing_List.xlsx", sheet_name='Output Table')
sspt_pricing_list_outputTable = sspt_pricing_list_outputTable.astype({'Product SKU':str},errors='raise')

sntc_pricing_list = pd.read_excel(f"{local_path}/Mapping files/4-SNTC_Princing_List.xlsx", sheet_name='Output')
sntc_pricing_list = sntc_pricing_list.astype({'Product SKU ':str},errors='raise')

htec_contract_types_htec = pd.read_excel(f"{local_path}/Mapping files/6-HTEC_Contract_Types.xlsx", sheet_name='HTEC')
htec_contract_types_htec = htec_contract_types_htec.astype({'GSP':str},errors='raise')

htec_contract_types_swss = pd.read_excel(f"{local_path}/Mapping files/6-HTEC_Contract_Types.xlsx", sheet_name='SWSS')
htec_contract_types_swss = htec_contract_types_swss.astype({'GSP':str},errors='raise')

combined_services = pd.read_excel(f"{local_path}/Mapping files/8-Combined_Services.xlsx", sheet_name='Sheet1')
combined_services = combined_services.astype({'Combined Services Service Level':str},errors='raise')

product_banding = pd.read_excel(f"{local_path}/Mapping files/9-PRODUCT_BANDING.xlsx", sheet_name='9-PRODUCT_BANDING')
product_banding = product_banding.astype({'INTERNAL_BE_PRODUCT_FAMILY':str},errors='raise')

expert_care_component_bands = pd.read_excel(f"{local_path}/Mapping files/7-Expert_Care_Components_Bands.xlsx")

# ------------------------------------------------------------------- Lists for Q&A
AC_NAME=[]
AC_ID=[]
IB_VALUE=[]
IB_COV=[]
MR=[]
SSPTOPP=[]
STOPP=[]
EXC=[]
SNT=[]
SN_LEGHT = []
ST_L2_LEGHT=[]
PACKAGE_INFO=[]

# ------------------------------------------------------------------- Accounts loop
    
for idx in range(len(fields_df)):
    req_type = fields_df['ID TYPE'][idx]
    name = re.sub('[^A-Za-z0-9\-]+', '', fields_df['Customer Name'][idx][0:15])
    theater = fields_df['Lvl1'][idx]
    party_ids = fields_df['cr_list'][idx]
    print(name)
    
    if req_type == 'SAV ID':
        ids = fields_df['sav_list'][idx]
        ids_smartsheet = [fields_df['SAV ID'][idx]]
        ids_smartsheet2 = ids
        print(ids)
        id_field = 'SAV ID'
        id_field_sw = 'SAV ID'
        id_flag_tac = 'SAV'
        renewals = renewals_s.copy()
        coverage = coverage_s.copy()
        subs = subs_s.copy()
        tac = tac_df.copy()
        id_field_tac = 'ID'
        
    elif req_type == 'GU ID':
        ids = fields_df['gu_list'][idx]
        ids_smartsheet = [fields_df['GU ID'][idx]]
        ids_smartsheet2 = ids
        print(ids)
        id_field = 'Best Site GU Party ID'
        id_field_sw = 'GU Party ID'
        id_flag_tac = 'GU'
        renewals = renewals_g.copy()
        coverage = coverage_g.copy()
        subs = subs_g.copy()
        tac = tac_df_gu.copy()
        id_field_tac = 'ID'
        
    elif req_type == 'CAV ID':
        cav_names["CAV ID"] = cav_names["CAV ID"].astype(str)
        cav_name_id = pd.DataFrame(data = {"CAV ID":fields_df[["cav_list"]].iloc[idx,0]}).merge(cav_names,how="innef")["CAV NAME"].to_list()
        ids = cav_name_id
        ids_smartsheet = [fields_df["CAV ID"][idx]]
        ids_smartsheet2 = fields_df['cav_list'][idx]
        print(ids)
        id_field = 'CAV - Account Name'
        id_field_sw = 'CAV -  Account Name'
        id_flag_tac = 'CAV'
        renewals = renewals_c.copy()
        coverage = coverage_c.copy()
        subs = subs_c.copy()
        tac = tac_df_cav.copy()
        id_field_tac = 'ID'
        
    elif req_type == 'CR Party ID':
        ids = fields_df['cr_list'][idx]
        ids_smartsheet = [fields_df['CR Party ID'][idx]]
        ids_smartsheet2 = ids
        print(ids)
        id_field = 'Best Site CR Party ID'
        id_field_sw = 'CR Party ID'
        id_flag_tac = 'GU'
        renewals = renewals_cr.copy()
        coverage = coverage_cr.copy()
        subs = subs_cr.copy()
        tac = tac_df_cr.copy()
        id_field_tac = 'PARTY ID'
    
    if req_type == "CAV ID":
        req_id = fields_df['CAV ID'][idx][0:6]
    else:
        req_id = ids[0]
    
    # ---------------------------------------------------------------------- Folders creation

    folder =  f"OP_{theater}_{req_type}_{req_id}_{name}_{str(date.year)}_{str(date.month)}_{str(date.day)}" # folder name
    folder_path = f"{current_path}/OP/{month}/{date}/{folder}"
    
    try:
        os.makedirs(f"{folder_path}/Data")
        os.makedirs(f"{folder_path}/Extracts")
        
    except: pass    
    
    folders_list.append(folder)
    folders_path_list.append(f'{folder_path}/{folder}.twb')
    
    # --------------------------------------------------------------------- Data transform and filtering
    
    smartsheet_filtered = fields_df[smartsheet_fields].iloc[[idx]]
    smartsheet_filtered = smartsheet_filtered.rename(columns={"Who should be notified on completion of Analysis":"Parties Active Collectors"})
    renew_filtered = renewals.query("`{}` in {}".format(id_field,ids))
    coverage_filtered  = coverage.query("`{}` in {}".format(id_field,ids))
    subs_filtered  = subs.query("`{}` in {}".format(id_field_sw,ids))
    
    if req_type == 'CAV ID':
        renew_filtered = cav_names.merge(renew_filtered,how="right",left_on="CAV NAME",right_on = "CAV - Account Name").drop(columns="CAV NAME")
        renew_filtered["CAV ID"] = renew_filtered["CAV ID"].astype(int)
        renew_filtered["Item Quantity"] = renew_filtered["Item Quantity"].astype(float)
        coverage_filtered = cav_names.merge(coverage_filtered,how="right",left_on="CAV NAME",right_on = "CAV - Account Name").drop(columns="CAV NAME")
        subs_filtered = cav_names.merge(subs_filtered,how="right",left_on="CAV NAME",right_on = "CAV -  Account Name").drop(columns="CAV NAME")
        
        
    dna_filtered = dna_df[dna_df['CUSTOMER_ID'].isin([int(ids) for ids in ids_smartsheet2])]
    dna_filtered = dna_filtered[dna_filtered['ACCOUNT_INDENTIFIER']==map_req_type.get(req_type)]
    
    telemetry_filtered = telemetry_df[telemetry_df['ID'].isin([int(ids) for ids in ids_smartsheet2])]
    telemetry_filtered = telemetry_filtered[telemetry_filtered['ACCOUNT_ID']==map_req_type.get(req_type)]
    parties_active_collectors = telemetry_filtered["Party ID"].unique().astype(str).tolist()
    parties_active_collectors = ",".join(parties_active_collectors)                                         
    
    smartsheet_filtered["Parties Active Collectors"][idx] = parties_active_collectors
    #telemetry_filtered = telemetry_df.query("`{}` in {}".format('Party ID',[int(i) for i in party_ids if i != '']))
    sntc_oppty_list.append(utils.SNTC_Oppty(renew_filtered,sntc_mapping))
    
    if req_type == 'CAV ID':
        ids = fields_df['cav_list'][idx]
    
    tac_filtered = tac.query("(`{}` in {}) and (FLAG == '{}')".format(id_field_tac,[int(i) for i in ids],id_flag_tac)) # id in TAC must be integer, not str
        
    print(len(renew_filtered))
    
    #renew_filtered.to_csv(f"{folder_path}/Data/IB/IB.csv",index=False)
    #coverage_filtered.to_csv(f"{folder_path}/Data/Coverage.csv",index=False)
    #subs_filtered.to_csv(f"{folder_path}/Data/SW.csv",index=False)
    dna_filtered.to_csv(f"{folder_path}/Data/DNA.csv",index=False)
        
    renew_filtered = utils.fill_nas(renew_filtered)
    coverage_filtered = utils.fill_nas(coverage_filtered)
    subs_filtered = utils.fill_nas(subs_filtered)
    
    if req_type == 'SAV ID':
        renew_filtered[['SAV ID','LDoS FY','Instance Shipped Fiscal Year','Contract Number']] = renew_filtered[['SAV ID','LDoS FY','Instance Shipped Fiscal Year','Contract Number']].fillna(0).apply(pd.to_numeric, downcast='integer')
    elif req_type == 'GU ID':
        renew_filtered[['Best Site GU Party ID','LDoS FY','Instance Shipped Fiscal Year','Contract Number']] = renew_filtered[['Best Site GU Party ID','LDoS FY','Instance Shipped Fiscal Year','Contract Number']].fillna(0).apply(pd.to_numeric, downcast='integer')
    elif req_type == 'CAV ID':
        #renew_filtered[['CX Customer ID','LDoS FY','Instance Shipped Fiscal Year','Contract Number']] = renew_filtered[['CX Customer ID','LDoS FY','Instance Shipped Fiscal Year','Contract Number']].fillna(0).apply(pd.to_numeric, downcast='integer')
        renew_filtered[['CAV ID','LDoS FY','Instance Shipped Fiscal Year','Contract Number']] = renew_filtered[['CAV ID','LDoS FY','Instance Shipped Fiscal Year','Contract Number']].fillna(0).apply(pd.to_numeric, downcast='integer')
    elif req_type == 'CR Party ID':
        renew_filtered[['Best Site CR Party ID','LDoS FY','Instance Shipped Fiscal Year','Contract Number']] = renew_filtered[['Best Site CR Party ID','LDoS FY','Instance Shipped Fiscal Year','Contract Number']].fillna(0).apply(pd.to_numeric, downcast='integer')
   
    renew_filtered['LDoS'] = pd.to_datetime(renew_filtered['LDoS'], errors='coerce').replace({pd.NaT:None})
    subs_filtered[['Contract Term End Quarter']] = subs_filtered[['Contract Term End Quarter']].fillna(0).apply(pd.to_numeric, downcast='integer')
    renew_filtered['Item Quantity'] = renew_filtered['Item Quantity'].astype(float)
    coverage_filtered['Item Quantity'] = coverage_filtered['Item Quantity'].astype(int)
    
    # --------------------------------------------------------------------- Template xml set up

    tree = ET.parse(tableau_template_name)
    
    for extract_path in tree.getroot().findall("./datasources/datasource/connection/named-connections/named-connection/connection[@class='textscan']"):
        #extract_name = extract_path.attrib['directory'].split('/')[-1]
        extract_path.attrib['directory'] = f"{folder_path}/Data"
    
    for extract_path in tree.getroot().findall("./datasources/datasource/connection/named-connections/named-connection/connection[@class='hyper']"):
        extract_name = extract_path.attrib['dbname'].split('/')[-1]
        extract_path.attrib['dbname'] = f"{folder_path}/Extracts/{extract_name}"
    
    with open (f"{folder_path}/{folder}.twb", "wb") as files :                                             
        tree.write(files)
        
    # --------------------------------------------------------------------- Package type
        
    if len(renew_filtered) > 0:
        if len(telemetry_filtered) == 0:
            type_list.append('Lite')
        else:
            type_list.append('Prime')
    elif len(renew_filtered) == 0:
        type_list.append('No Report')
    
    # --------------------------------------------------------------------- Extracts creation

    utils.create_extract(name='Smartsheet',columns=utils.get_schema(table='smartsheet',id_type=req_type),df=smartsheet_filtered,path=folder_path)
    #utils.create_extract(name='IB',columns=utils.get_schema(table='ib',id_type=req_type),df=renew_filtered,path=folder_path)
    #utils.create_extract(name='Coverage',columns=utils.get_schema(table='coverage',id_type=req_type),df=coverage_filtered,path=folder_path)
    #utils.create_extract(name='SW',columns=utils.get_schema(table='sw',id_type=req_type),df=subs_filtered,path=folder_path)
    #utils.create_extract(name='TAC',columns=utils.get_schema(table='tac2',id_type=req_type),df=tac_filtered,path=folder_path)    
    #utils.create_extract(name='CIR',columns=utils.get_schema(table='cir',id_type=req_type),df=telemetry_filtered.drop(columns=['ACCOUNT_ID','ID']),path=folder_path)


    # --------------------------------------------------------------------- CSV creation for Tableau
    ### ---------------------------------------  Install Base merge 

    renew_filtered = utils.set_datasource(renew_filtered,'ib',f"{folder_path}/Data/1-Installed Base (2).csv",contract_types_list,
                    success_track_pricing_list,sspt_pricing_list_eligibleSSPT,
                    sspt_pricing_list_outputTable,sntc_pricing_list,htec_contract_types_htec,
                    htec_contract_types_swss,combined_services,product_banding)

    ### ---------------------------------------  SNTC Collector
    telemetry_filtered = utils.set_datasource(telemetry_filtered,'cir',f"{folder_path}/Data/3-SNTC Collector (2).csv",contract_types_list,
                    success_track_pricing_list,sspt_pricing_list_eligibleSSPT,
                    sspt_pricing_list_outputTable,sntc_pricing_list,htec_contract_types_htec,
                    htec_contract_types_swss,combined_services,product_banding)

    ### ---------------------------------------  Coverage
    coverage_filtered = utils.set_datasource(coverage_filtered,'coverage',f"{folder_path}/Data/Coverage.csv",contract_types_list,
                    success_track_pricing_list,sspt_pricing_list_eligibleSSPT,
                    sspt_pricing_list_outputTable,sntc_pricing_list,htec_contract_types_htec,
                    htec_contract_types_swss,combined_services,product_banding)


    ### ---------------------------------------  SW
    subs_filtered = utils.set_datasource(subs_filtered,'sw',f"{folder_path}/Data/SW.csv",contract_types_list,
                    success_track_pricing_list,sspt_pricing_list_eligibleSSPT,
                    sspt_pricing_list_outputTable,sntc_pricing_list,htec_contract_types_htec,
                    htec_contract_types_swss,combined_services,product_banding)

    ### ---------------------------------------  TAC
    tac_filtered = utils.set_datasource(tac_filtered,'tac',f"{folder_path}/Data/6-TAC.csv",contract_types_list,
                    success_track_pricing_list,sspt_pricing_list_eligibleSSPT,
                    sspt_pricing_list_outputTable,sntc_pricing_list,htec_contract_types_htec,
                    htec_contract_types_swss,combined_services,product_banding)

    ## ---------------------------------------  Expert Care

    expert_care_component_bands.to_csv(f"{folder_path}/Data/7-Expert_Care_Components_Bands.csv", index=False)

    # --------------------------------------------------------------------- Calculating Q&A values

    ib_value = utils.ib_value_validation(utils.IB_attributes(renew_filtered))
    ib_covered = utils.ib_covered_validation(utils.IB_attributes(renew_filtered))
    mayor_rw = utils.rw_validation(utils.IB_attributes(renew_filtered))
    sspt_opp = utils.oppty_validation(utils.SSPT_Oppty(renew_filtered))
    st_opp = utils.oppty_validation(utils.ST_Oppty(renew_filtered))
    exp_care = utils.oppty_validation(utils.expert_care_verification(expert_care_component_bands))
    sntc_val = utils.oppty_validation(utils.smartnet_verification(renew_filtered))
    snl = utils.lenght_validation(utils.smartnet_total_care_NBD_list_price(renew_filtered)[1])
    stl = utils.lenght_validation(utils.estimated_list_price(renew_filtered)[1])
    smart_sheet = fields_df[fields_df['Customer Name'] == smartsheet_filtered['Customer Name'].iloc[0]]
    pack = utils.smartsheet_len_info(smart_sheet)

    
    # --------------------------------------------------------------------- Negative Oppty Correction

    if (sspt_opp == 'Negative Value') or (st_opp == 'Negative Value'):
        print(utils.SSPT_Oppty(renew_filtered))
        renew_filtered['Annualized Extended Contract Line List USD Amount'] = renew_filtered['Default Service List Price USD']
        renew_filtered.to_csv(f"{folder_path}/Data/1-Installed Base (2).csv", index=False)
        sspt_opp_value = utils.SSPT_Oppty(renew_filtered)
        st_oppty_value = utils.ST_Oppty(renew_filtered)
        if sspt_opp == 'Negative Value':
            print(bcolors.FAIL + bcolors.BOLD + 'Corrected SSPT oppty values: ' + str(sspt_opp_value)+ bcolors.ENDC)
        if st_opp == 'Negative Value':
            print(bcolors.FAIL + bcolors.BOLD + 'Corrected ST oppty values: ' + str(st_oppty_value)+ bcolors.ENDC)
        sspt_opp = utils.oppty_validation(utils.SSPT_Oppty(renew_filtered))
        st_opp = utils.oppty_validation(utils.ST_Oppty(renew_filtered))

    
    # ---------------------------------------------------------------Filling lists for Q&A dataframe

    AC_NAME.append(name)
    AC_ID.append(req_id)
    IB_VALUE.append(ib_value)
    IB_COV.append(ib_covered)
    MR.append(mayor_rw)    
    EXC.append(exp_care)
    SNT.append(sntc_val)    
    SSPTOPP.append(sspt_opp)
    STOPP.append(st_opp)
    SN_LEGHT.append(snl)
    ST_L2_LEGHT.append(stl) 
    PACKAGE_INFO.append(pack)

folders_list

In [ ]:
 # --------------------------------------------------------------------- Generating DF needed for Q&A

validation_data = pd.DataFrame(columns=['Account Name', 'ID', 'IB Value', 'IB Covered', 'Mayor Renewal', 'SSPT Opportunity', 'ST Opportunity', 'Expert Care validation', 'Smartnet validation'])
validation_data['Account Name']=AC_NAME
validation_data['ID']=AC_ID
validation_data['IB Value']=IB_VALUE
validation_data['IB Covered']=IB_COV
validation_data['Mayor Renewal']=MR
validation_data['SSPT Opportunity']=SSPTOPP
validation_data['ST Opportunity']=STOPP
validation_data['Expert Care validation']=EXC
validation_data['Smartnet validation']=SNT
validation_data['Smartnet value lenght'] = SN_LEGHT
validation_data['Success_L2 value lenght'] = ST_L2_LEGHT
validation_data['Package Info'] = PACKAGE_INFO
validation_data['File path'] = folders_path_list


validation_data.loc[:,'Account Name':'Package Info'].style.applymap(lambda x: utils.color_qa(x))

In [ ]:
# ----------------------------------------------------------------Oppening defective reports

paths = validation_data[(validation_data.loc[:,'IB Value':'Package Info'] != 'Correct').any(1)]['File path'].apply(os.startfile)

In [ ]:
importlib.reload(utils)

# Tableau Server Automatic Workbook Publishing
## Do not use before asking ;)

In [ ]:
folders_id = {
    "US COMMERCIAL": "fd7772ee-058d-463a-bd0a-4f65542801a4", # Americas
    "GLOBAL ENTERPRISE SEGMENT": "e57405f3-ca6c-4bc2-ad00-6e9e4eaddca3",
    "LATIN AMERICA": "11411241-08ec-488e-b5e5-b858051464a5",
    "CANADA": "2c7d703c-ccd7-4ebb-9a80-493efec42b37",
    "AMERICAS_SP": "001fdd45-0e78-4c94-8529-f541a58903b0",
    "US PS Market Segment": "bc2b9643-6dab-4659-967c-4c7faa1c9fbd",
    "ANZ AREA": "dc644422-3a51-4940-9e0c-5b9eeba4e938", # APJC
    "ASEAN_AREA": "486500cf-bab3-42f2-8cf6-912e0db03056",
    "GREATER_CHINA": "48eaf322-742a-4b72-a750-fe87a18a3e46",
    "INDIA_AREA": "3eda13f8-ce22-40fc-89e1-cca9f69318ee",
    "JAPAN__": "18c5b29f-d451-4546-b924-0f9964d98c95",
    "ROK_AREA": "0f98151c-0030-4f4b-90fc-a785b444cea1",
    "APJ_SP": "517cddf4-032f-4dc0-9569-23ce801ebd33",
    "EMEAR_GERMANY": "d11b1067-045d-42cb-a4c8-67805e4523a4", # EMEAR
    "EMEAR_SP": "35a5ac5e-1654-421e-bb75-d8c95a9408e3",
    "EMEAR-SOUTH": "7e91e72f-18bf-4b3b-8b72-7b964140743b",
    "EMEAR-NORTH": "34b13798-5c47-4bcf-b3d1-917d802d511f",
    "EMEAR-UKI": "268fdf46-d3ee-4623-ace9-97bb32ccc328",
    "EMEAR_MEA": "95dcde09-a6fc-4fb1-b7a8-69e1dc4775e8",
    "EMEAR-CENTRAL": "038970b3-8ea2-4a9a-86e8-6bd470350656"
}

fields_df2 = fields_df.copy()
fields_df2['folder_id'] = fields_df2['Lvl2 (Region)'].apply(lambda x : folders_id.get(x,''))
fields_df2['project_name'] = folders_list
fields_df2['project_url'] = fields_df2['project_name'].apply(lambda x: utils.get_url(x))
workbook_names = list(fields_df2['project_name'].unique())
fields_df2[['Customer Name','Lvl2 (Region)','folder_id','project_name','project_url']]

In [ ]:
# --------------------------------------------------------- Connection with tableau server

import os
import datetime

from tableau_api_lib import TableauServerConnection
from tableau_api_lib.utils import querying
import time

current_path = os.getcwd().replace("\\","/")

month = datetime.datetime.today().strftime("%B")

config = {
    "tableau_server": {
        'server': 'https://cx-tableau-stage.cisco.com',
        'api_version': '3.13',
        'personal_access_token_name': personal_access_token_name,
        'personal_access_token_secret': personal_access_token_secret,
        'site_name': 'Compass',
        'site_url': 'Compass'
    }
}

conn = TableauServerConnection(config,env='tableau_server')

for idx in range(len(fields_df2)):
    conn.sign_in()
    project = fields_df2['project_name'][idx]
    name = fields_df2['Customer Name'][idx]

# ----------------------------------------------------------------Publishing reports   

    try: 

        response = conn.publish_workbook(
            project_id=fields_df2['folder_id'][idx],
            workbook_file_path=f"{current_path}/OP/{month}/{date}/{project}/{project}.twbx",
            workbook_name="{}".format(project),
            workbook_views_to_hide=['QA','Package Info'],
            hide_view_flag=True
        )

        if (response.status_code != 201):    
            print(bcolors.FAIL + bcolors.BOLD + "Estimator has an error and has not been published"+ bcolors.ENDC)
            raise AssertionError()

        else : print(f"The {name} Estimator has been published")

    except Exception as e: 
        print(bcolors.FAIL + bcolors.BOLD + str(e)+ bcolors.ENDC)
        pass

    time.sleep(3)
    conn.sign_out()


# -------------------------------------------------------------------    Q&A of URL's


signin={"credentials": {"personalAccessTokenName": personal_access_token_name, 
                        "personalAccessTokenSecret": personal_access_token_secret,
                        "site": {"contentUrl": "Compass"}}}

auth = requests.post('https://cx-tableau-stage.cisco.com/api/3.13/auth/signin', headers={'Content-Type' : 'application/json'}, data=json.dumps(signin))
auth = ET.fromstring(auth.text)
auth_token = auth[0].attrib['token']
user_id = auth[0][1].attrib['id']
site_id = auth[0][0].attrib['id']

print("API connected")

workbooks = requests.get(f'https://cx-tableau-stage.cisco.com/api/3.13/sites/{site_id}/workbooks?filter=createdAt:gte:{datetime.datetime.strftime(date,"%Y-%m-%d")}T00:00:00Z', headers={'X-Tableau-Auth':auth_token})
workbook_parser = ET.fromstring(workbooks.text)

published_workbooks = [workbook_parser[1][i].attrib['name'] for i in range(len(workbook_parser[1][:]))]


wrong_url = []

for workbook in workbook_names:
    if workbook not in published_workbooks:
        wrong_url.append(workbook)

if len(wrong_url)==0:
    print('Everything OK')
else:
    print(bcolors.FAIL + bcolors.BOLD + "Please review te following reports: \n"+ bcolors.ENDC)
    print(wrong_url)
    
        

In [ ]:
for link in fields_df2['project_url']:
    webbrowser.open(link)
    print(link)

In [ ]:
for op_type in type_list:
    print(op_type)

In [ ]:
for oppty in sntc_oppty_list:
    if oppty == 'N/A':
        print(oppty)
    else:
        print("%.1f" % oppty)